In [1]:
from pyspark.sql import SQLContext

In [2]:
sqlsc = SQLContext(sc)

In [4]:
df = sqlsc.read.load('file:///home/cloudera/Downloads/big-data/game-clicks.csv',
                          format='com.databricks.spark.csv',
                          header='true',
                          inferSchema='true')

In [5]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- clickId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- isHit: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- teamLevel: integer (nullable = true)



In [6]:
df.count()

755806

In [7]:
df.show(5)

+--------------------+-------+------+-------------+-----+------+---------+
|           timestamp|clickId|userId|userSessionId|isHit|teamId|teamLevel|
+--------------------+-------+------+-------------+-----+------+---------+
|2016-05-26 15:06:...|    105|  1038|         5916|    0|    25|        1|
|2016-05-26 15:07:...|    154|  1099|         5898|    0|    44|        1|
|2016-05-26 15:07:...|    229|   899|         5757|    0|    71|        1|
|2016-05-26 15:07:...|    322|  2197|         5854|    0|    99|        1|
|2016-05-26 15:07:...|     22|  1362|         5739|    0|    13|        1|
+--------------------+-------+------+-------------+-----+------+---------+
only showing top 5 rows



In [9]:
df.select("userId", "teamLevel").show(5)

+------+---------+
|userId|teamLevel|
+------+---------+
|  1038|        1|
|  1099|        1|
|   899|        1|
|  2197|        1|
|  1362|        1|
+------+---------+
only showing top 5 rows



In [10]:
df.filter(df["teamLevel"] > 1).select("userId", "teamLevel").show(5)

+------+---------+
|userId|teamLevel|
+------+---------+
|  1513|        2|
|   868|        2|
|  1453|        2|
|  1282|        2|
|  1473|        2|
+------+---------+
only showing top 5 rows



In [11]:
df.groupBy("isHit").count().show()

+-----+------+
|isHit| count|
+-----+------+
|    0|672423|
|    1| 83383|
+-----+------+



In [12]:
from pyspark.sql.functions import *

In [13]:
df.select(mean('isHit'), sum('isHit')).show()

+------------------+----------+
|        avg(isHit)|sum(isHit)|
+------------------+----------+
|0.1103232840173272|     83383|
+------------------+----------+



In [19]:
df2 = sqlsc.read.load('file:///home/cloudera/Downloads/big-data/ad-clicks.csv',
                          format='com.databricks.spark.csv',
                          header='true',
                          inferSchema='true')

In [20]:
df2.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- txId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- adId: integer (nullable = true)
 |-- adCategory: string (nullable = true)



In [21]:
merge = df.join(df2, 'userId')

In [22]:
merge.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- clickId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- isHit: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- teamLevel: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- txId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- adId: integer (nullable = true)
 |-- adCategory: string (nullable = true)



In [23]:
merge.show(5)

+------+--------------------+-------+-------------+-----+------+---------+--------------------+-----+-------------+------+----+----------+
|userId|           timestamp|clickId|userSessionId|isHit|teamId|teamLevel|           timestamp| txId|userSessionId|teamId|adId|adCategory|
+------+--------------------+-------+-------------+-----+------+---------+--------------------+-----+-------------+------+----+----------+
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 01:40:...|23669|        23626|   142|  27|     games|
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 09:24:...|24122|        23626|   142|   4|     games|
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 17:21:...|24659|        23626|   142|  22| computers|
|   231|2016-06-08 00:45:...| 376796|        23626|    0|   142|        4|2016-06-08 23:34:...|25076|        23626|   142|  21|    movies|
|   231|2016-06-08 00:45:..